In [1]:
#importing libraries
import numpy as np
import pandas as pd
import datetime as dt # Datetime
import json # library to handle JSON files
import hmac
from geopy.geocoders import Nominatim
from geopy.distance import vincenty
# import k-means from clustering stage
from sklearn.cluster import KMeans

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

print('Libraries imported.')

Libraries imported.


In [2]:
#Loading dataset from web
df_man = pd.read_excel("https://www1.nyc.gov/assets/finance/downloads/pdf/rolling_sales/rollingsales_manhattan.xls")
df_bronx = pd.read_excel("https://www1.nyc.gov/assets/finance/downloads/pdf/rolling_sales/rollingsales_bronx.xls")
df_brooklyn = pd.read_excel("https://www1.nyc.gov/assets/finance/downloads/pdf/rolling_sales/rollingsales_brooklyn.xls")
df_queens = pd.read_excel("https://www1.nyc.gov/assets/finance/downloads/pdf/rolling_sales/rollingsales_queens.xls")

In [3]:
df_bronx

,Bronx Rolling Sales File. All Sales From November 2019 - October 2020,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,PTS Sales as of 11/12/2020,Unnamed: 20
0,"For sales prior to the Final, Neighborhood Nam...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Building Class Category is based on Building C...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Note: Condominium and cooperative sales are on...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,APARTMENT NUMBER,...,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
4,2,BATHGATE,01 ONE FAMILY DWELLINGS,1,3029,36,NaN,A5,418 EAST 180 STREET,NaN,...,1,0,1,1568,1526,1899,1,A5,0,2020-07-09 00:00:00
5,2,BATHGATE,01 ONE FAMILY DWELLINGS,1,3030,65,NaN,A1,4455 PARK AVENUE,NaN,...,1,0,1,1622,1587,1899,1,A1,458000,2020-01-31 00:00:00
6,2,BATHGATE,01 ONE FAMILY DWELLINGS,1,3030,67,NaN,A1,4451 PARK AVENUE,NaN,...,1,0,1,1646,1495,1899,1,A1,475000,2020-07-23 00:00:00
7,2,BATHGATE,01 ONE FAMILY DWELLINGS,1,3030,67,NaN,A1,4451 PARK AVENUE,NaN,...,1,0,1,1646,1495,1899,1,A1,275000,2020-03-16 00:00:00
8,2,BATHGATE,01 ONE FAMILY DWELLINGS,1,3039,29,NaN,A9,2327 WASHINGTON AVENUE,NaN,...,1,0,1,1095,1260,1910,1,A9,110000,2019-12-06 00:00:00
9,2,BATHGATE,01 ONE FAMILY DWELLINGS,1,3039,29,NaN,A9,2327 WASHINGTON AVE,NaN,...,1,0,1,1095,1260,1910,1,A9,165000,2020-03-02 00:00:00


## Cleaning dataset


In [4]:
a1 = df_man.iloc[:,1:]
a1 = a1.drop(df_man.index[:3])
a1 = a1.rename(columns=a1.iloc[0]).drop(a1.index[0])
a1 = a1.drop(columns=["BUILDING CLASS CATEGORY","TAX CLASS AT PRESENT","BLOCK","LOT","BUILDING CLASS AT PRESENT","SALE DATE","BUILDING CLASS AT TIME OF SALE","EASE-MENT"])
a1 = a1.rename(columns = {"SALE PRICE":"saleprice"})
a1 = a1[a1['saleprice'] > 10000]  

In [5]:
a2 = df_bronx.iloc[:,1:]
a2 = a2.drop(df_bronx.index[:3])
a2 = a2.rename(columns=a2.iloc[0]).drop(a2.index[0])
a2 = a2.drop(columns=["BUILDING CLASS CATEGORY","TAX CLASS AT PRESENT","BLOCK","LOT","BUILDING CLASS AT PRESENT","SALE DATE","BUILDING CLASS AT TIME OF SALE","EASE-MENT"])
a2= a2.rename(columns = {"SALE PRICE":"saleprice"})
a2 = a2[a2['saleprice'] > 10000]  

In [6]:
a3 = df_brooklyn.iloc[:,1:]
a3 = a3.drop(df_brooklyn.index[:3])
a3 = a3.rename(columns=a3.iloc[0]).drop(a3.index[0])
a3 = a3.drop(columns=["BUILDING CLASS CATEGORY","TAX CLASS AT PRESENT","BLOCK","LOT","BUILDING CLASS AT PRESENT","SALE DATE","BUILDING CLASS AT TIME OF SALE","EASE-MENT"])
a3 = a3.rename(columns = {"SALE PRICE":"saleprice"})
a3 = a3[a3['saleprice'] > 10000]  

In [7]:
a4 = df_queens.iloc[:,1:]
a4 = a4.drop(df_queens.index[:3])
a4 = a4.rename(columns=a4.iloc[0]).drop(a4.index[0])
a4 = a4.drop(columns=["BUILDING CLASS CATEGORY","TAX CLASS AT PRESENT","BLOCK","LOT","BUILDING CLASS AT PRESENT","SALE DATE","BUILDING CLASS AT TIME OF SALE","EASE-MENT"])
a4 = a4.rename(columns = {"SALE PRICE":"saleprice"})
a4 = a4[a4['saleprice'] > 10000]  

In [8]:
df = pd.concat([a1,a2,a3,a4])
df = df.dropna(subset=["saleprice","ADDRESS"])


In [9]:
import string
df.ADDRESS = df.ADDRESS.str.lstrip(string.digits)
df.ADDRESS = df.ADDRESS.str.rstrip(string.digits)
df.ADDRESS = df.ADDRESS.str.replace("-","")
df.ADDRESS = df.ADDRESS.str.replace("N/A","")

In [10]:
df.ADDRESS = df.ADDRESS.str.split(",").str[0]


In [11]:
#Filtering address and saleprice alone:
df1 = df["ADDRESS"]
df2 = df["saleprice"]
df3 = pd.concat([df1, df2], axis=1, sort=False)
df3.ADDRESS = df3.ADDRESS.str.lstrip(string.digits)
df3["ADDRESS"].value_counts()


 PARK AVENUE              301
 WEST END AVENUE          259
 BROADWAY                 211
 FIFTH AVENUE             180
 44TH DRIVE               153
 CENTRAL PARK WEST        138
 39TH AVENUE              136
 RIVERSIDE DRIVE          135
 OCEAN PARKWAY            127
 OCEAN AVENUE             125
 WEST 61ST STREET         121
 82ND STREET              112
 LEONARD STREET           110
 EAST 11TH STREET         109
 83RD STREET              101
 EAST 72ND STREET         100
 METROPOLITAN AVENUE       99
 WILLOUGHBY STREET         99
 5 AVENUE                  97
 SANFORD AVENUE            88
 CENTRAL PARK SOUTH        86
 GRAND CENTRAL PARKWAY     86
 EAST 57TH STREET          84
 NEVINS STREET             84
 LEXINGTON AVENUE          83
 EAST 79TH STREET          83
 EAST 7TH STREET           82
 CLINTON AVENUE            76
 SHORE ROAD                76
 UNION STREET              76
                         ... 
 BEACH 219TH STREET         1
 FLATLANDS 7TH STREET       1
 CEDAR ST 

In [12]:
df3.sort_values(by=['saleprice'],ascending=[False],inplace=True)
df3

,ADDRESS,saleprice
1265,/434 5 AVENUE,978090439
4877,MADISON AVENUE,900000000
4895,PARK AVENUE,616755253
5656,3 AVENUE,451250000
1224,BROADWAY,435000000
7112,BROADWAY,382000000
6465,EAST 28TH STREET,380600000
12564,22ND STREET,369300000
12562,21ST STREET,369300000
12548,REVIEW AVENUE,369300000


In [13]:
x = []
for i in df3["ADDRESS"]:
    i = i + ",NY"
    x.append(i)
x = pd.DataFrame(x)
x = x.rename(columns = {0:"ADDRESS1"})
df4 = pd.concat([df3,x],axis=1,join="inner")
df4 = df4.drop(columns=["ADDRESS"])
df4 = df4.rename(columns={"ADDRESS1":"ADDRESS"})
df3=df4
print(df3)

       saleprice                  ADDRESS
1265   978090439           PARK AVENUE,NY
4877   900000000          LINCOLN ROAD,NY
4895   616755253                WARREN,NY
5656   451250000      WEST 90TH STREET,NY
1224   435000000           PARK AVENUE,NY
7112   382000000           48TH AVENUE,NY
6465   380600000      WEST 42ND STREET,NY
12564  369300000          220TH STREET,NY
12562  369300000           88TH STREET,NY
12548  369300000          GRAND AVENUE,NY
6365   350000000      WEST 28TH STREET,NY
6373   350000000         WARREN STREET,NY
6371   350000000         BAY 16 STREET,NY
6370   350000000          LENOX AVENUE,NY
6369   350000000      EAST 29TH STREET,NY
6368   350000000           JANE STREET,NY
6367   350000000        PULASKI STREET,NY
6366   350000000       WEST 142 STREET,NY
6372   350000000         FULTON STREET,NY
12567  337534524        SEVENTH AVENUE,NY
12568  337534524             42ND ROAD,NY
12569  337534524       CRAWFORD AVENUE,NY
12570  337534524           55TH AV

In [14]:
df3 = df3.groupby('ADDRESS').filter(lambda x : (x['ADDRESS'].count()>=15).any())
df3


,saleprice,ADDRESS
1265,978090439,"PARK AVENUE,NY"
5656,451250000,"WEST 90TH STREET,NY"
1224,435000000,"PARK AVENUE,NY"
7112,382000000,"48TH AVENUE,NY"
6465,380600000,"WEST 42ND STREET,NY"
12562,369300000,"88TH STREET,NY"
12548,369300000,"GRAND AVENUE,NY"
6365,350000000,"WEST 28TH STREET,NY"
6373,350000000,"WARREN STREET,NY"
6370,350000000,"LENOX AVENUE,NY"


## creating final filtered dataset with street name and the average price of homes in the street

In [15]:
df3_sum = df3.groupby("ADDRESS").sum().reset_index()
df3_sum.size

1200

In [16]:
df3_count = df3.groupby("ADDRESS").agg(["count"]).reset_index()
df3_count.columns
df3_count.columns = df3_count.columns.get_level_values(0)
df3_count.rename(columns = {'saleprice':'count'}, inplace=True)
df3_count.size

1200

In [17]:
result = pd.concat([df3_sum,df3_count], axis=1, join='inner')
result
result_1 = pd.DataFrame()
result_1['Average'] = result['saleprice']/result['count']

In [18]:
temp = result['ADDRESS']
temp = temp.iloc[:,0]
result_1['Address'] = temp
result_1 = result_1[['Address','Average']]
final = result_1["Address"]

In [19]:
result_1

,Address,Average
0,"1 AVENUE,NY",2.201636e+06
1,"102ND STREET,NY",6.060346e+05
2,"103RD STREET,NY",1.394267e+06
3,"107TH STREET,NY",7.456809e+05
4,"108TH STREET,NY",7.949143e+05
5,"10TH AVENUE,NY",2.050444e+06
6,"10TH STREET,NY",1.264963e+06
7,"11TH AVENUE,NY",1.057836e+06
8,"11TH STREET,NY",2.893165e+06
9,"12TH AVENUE,NY",9.901825e+05


In [20]:
final

0                 1 AVENUE,NY
1             102ND STREET,NY
2             103RD STREET,NY
3             107TH STREET,NY
4             108TH STREET,NY
5              10TH AVENUE,NY
6              10TH STREET,NY
7              11TH AVENUE,NY
8              11TH STREET,NY
9              12TH AVENUE,NY
10             12TH STREET,NY
11             13TH AVENUE,NY
12             13TH STREET,NY
13            146TH STREET,NY
14            147TH STREET,NY
15            149TH STREET,NY
16             14TH AVENUE,NY
17             14TH STREET,NY
18            150TH STREET,NY
19            159TH STREET,NY
20             15TH AVENUE,NY
21             15TH STREET,NY
22            160TH STREET,NY
23            162ND STREET,NY
24            163RD STREET,NY
25            165TH STREET,NY
26            166TH STREET,NY
27            167TH STREET,NY
28            168TH STREET,NY
29            169TH STREET,NY
                ...          
570       WEST 82ND STREET,NY
571       WEST 83RD STREET,NY
572       

In [21]:
from geopy.geocoders import Nominatim
geolocator = Nominatim()
lati=[]
longi=[]
for i in final:
    try:
        location = geolocator.geocode(i)
        a = print((location.latitude, location.longitude))
        lat=location.latitude
        long=location.longitude
        lati.append(lat)
        longi.append(long)    
    except:
        pass;

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  from ipykernel import kernelapp as app


(40.7704527, -73.9540656)
(42.7562532, -73.6796191)
(40.7961299, -73.9614554)
(42.7634829, -73.6759243)
(40.7427729, -73.8546545)
(40.72467, -73.690318)
(40.728657, -73.640106)
(40.723987, -73.690423)
(40.7303915, -73.6398637)
(40.849118, -73.651344)
(40.7329321, -73.6355109)
(40.7889086, -73.1444946)
(40.73417, -73.6375297)
(40.6817838, -73.7954738)
(40.6829887, -73.7950648)
(40.7626228, -73.816153)
(40.848252, -73.649932)
(40.7380466, -73.9963408)
(40.7441393, -73.8180032)
(40.7406449, -73.8092648)
(40.847564, -73.644145)
(40.736402, -73.642619)
(40.7388763, -73.8084461)
(40.740457, -73.806461)
(40.740419, -73.805523)
(40.7873583, -73.7976726)
(40.7871098, -73.7967237)
(40.8357244, -73.9212797)
(40.8405084, -73.9402259)
(40.71047, -73.793604)
(40.8671914, -73.0181653)
(40.906445, -73.54766)
(40.7477425, -73.7965381)
(40.7015413, -73.7844124)
(40.6639191, -73.7705669)
(40.846956, -73.647188)
(40.906319, -73.548551)
(40.620671, -73.990414)
(40.7410394, -73.9978652)
(40.8591154, -73.934

(40.713715, -73.990173)
(40.785426, -73.504397)
(40.7188724, -73.97549)
(40.8223126, -73.8191615)
(40.6700628, -73.9539342)
(40.725999, -73.950029)
(40.724294, -73.9935159)
(42.7976632, -73.9527343)
(42.7976632, -73.9527343)
(42.8021386, -73.9528461)
(40.5827089, -73.8927536)
(40.7002809, -73.9411682)
(40.8093976, -73.8803468)
(40.672011, -73.679744)
(42.505816, -73.783553)
(42.7163045, -73.7201635)
(40.709426, -74.0065858)
(43.030515, -76.153397)
(40.698186, -73.619943)
(40.6895761, -73.9222019)
(40.699743, -73.488033)
(40.7643796, -73.97337386360579)
(41.0984113, -73.9204777)
(40.7181616, -73.9937839)
(44.4231033, -73.6345821)
(40.890812, -73.0370322)
(40.723795, -74.000367)
(40.733293, -73.9402955)
(40.7051446, -73.6218734)
(40.761801, -73.341836)
(40.8095665, -73.8802285)
(40.6859193, -73.915852)
(40.692358, -73.911053)
(40.666415, -73.667948)
(40.733742, -73.499338)
(40.684903, -73.53171)
(40.663482, -73.703814)
(41.0896983, -73.9286681)
(40.710912, -73.601182)
(43.128051, -75.225

#from geopy.geocoders import Nominatim
geolocator = Nominatim()
lati=[]
longi=[]
for i in final:
    location = geolocator.geocode(i)
    a = print((location.latitude, location.longitude))
    lat=location.latitude
    long=location.longitude
    lati.append(lat)
    longi.append(long)    

In [22]:
lati_temp=pd.DataFrame(lati)
longi_temp =pd.DataFrame(longi)
lati_temp=lati_temp.rename(columns={0:"Latitude"})
longi_temp=longi_temp.rename(columns={0:"Longitude"})


In [23]:
result_1 = pd.concat([result_1, lati_temp], axis=1)
result_1 = pd.concat([result_1, longi_temp], axis=1)

In [24]:
result_1=result_1.rename(columns={"Address":"Street","Average":"price"})

In [25]:
result_1=result_1.dropna()

In [26]:
df = result_1
df=df.dropna()

In [27]:
city = 'New York City, USA'

geolocator = Nominatim()
location = geolocator.geocode(city)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Newyork City are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinate of Newyork City are 40.7127281, -74.0060152.


In [28]:
!pip install folium
import folium
map_nyc = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, price, street in zip(df['Latitude'], df['Longitude'], df['price'], df['Street']):
    label = '{}, {}'.format(street, price)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_nyc)  
    
map_nyc
#marks represents the street.

     |████████████████████████████████| 102kB 9.1MB/s ta 0:00:011


In [29]:
#Inserting foursquareID
CLIENT_ID = 'AJKVZBLUKZI33L4NRPA5LYG0XWGUCYA0J2ER2A5JQIBI0WSU' # Foursquare ID
CLIENT_SECRET = 'NLUTWJCFCZHW0QYFSUAVZ1A0EP4GV4CBIO0KGMPJEW0FI4JS' # Foursquare Secret
VERSION = '20181206' # Foursquare API version


In [30]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Street', 
                  'Street Latitude', 
                  'Street Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [31]:
# creating a new dataframe called location_venues with each location and display it .
location_venues = getNearbyVenues(names=df['Street'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

 1 AVENUE,NY
 102ND STREET,NY
 103RD STREET,NY
 107TH STREET,NY
 108TH STREET,NY
 10TH AVENUE,NY
 10TH STREET,NY
 11TH AVENUE,NY
 11TH STREET,NY
 12TH AVENUE,NY
 12TH STREET,NY
 13TH AVENUE,NY
 13TH STREET,NY
 146TH STREET,NY
 147TH STREET,NY
 149TH STREET,NY
 14TH AVENUE,NY
 14TH STREET,NY
 150TH STREET,NY
 159TH STREET,NY
 15TH AVENUE,NY
 15TH STREET,NY
 160TH STREET,NY
 162ND STREET,NY
 163RD STREET,NY
 165TH STREET,NY
 166TH STREET,NY
 167TH STREET,NY
 168TH STREET,NY
 169TH STREET,NY
 16TH AVENUE,NY
 16TH STREET,NY
 171ST STREET,NY
 172ND STREET,NY
 177TH STREET,NY
 17TH AVENUE,NY
 17TH STREET,NY
 18TH AVENUE,NY
 18TH STREET,NY
 190TH STREET,NY
 192ND STREET,NY
 19TH AVENUE,NY
 19TH STREET,NY
 2 AVENUE,NY
 2 STREET,NY
 206TH STREET,NY
 20TH AVENUE,NY
 214TH STREET,NY
 215TH STREET,NY
 21ST AVENUE,NY
 21ST STREET,NY
 223RD STREET,NY
 22ND AVENUE,NY
 23RD AVENUE,NY
 23RD STREET,NY
 24TH AVENUE,NY
 24TH ROAD,NY
 25TH AVENUE,NY
 26TH AVENUE,NY
 27TH STREET,NY
 28TH AVENUE,NY
 28TH ROA

 PROSPECT PLACE,NY
 PULASKI STREET,NY
 PUTNAM AVENUE,NY
 QUEENS BOULEVARD,NY
 QUEENS PLAZA SOUTH,NY
 QUINCY STREET,NY
 RALPH AVENUE,NY
 RECTOR PLACE,NY
 RIVER TERRACE,NY
 RIVERSIDE BLVD,NY
 RIVERSIDE BOULEVARD,NY
 RIVERSIDE DRIVE,NY
 RIVINGTON STREET,NY
 ROGERS AVENUE,NY
 ROOSEVELT AVENUE,NY
 RUGBY ROAD,NY
 RUTLAND ROAD,NY
 SACKETT STREET,NY
 SANFORD AVENUE,NY
 SCHERMERHORN STREET,NY
 SECOND AVENUE,NY
 SEVENTH AVENUE,NY
 SHORE ROAD,NY
 SMITH STREET,NY
 SOUTH STREET,NY
 SPRING STREET,NY
 ST MARKS AVENUE,NY
 ST MARKS PLACE,NY
 ST NICHOLAS AVENUE,NY
 ST. JOHNS PLACE,NY
 STATE STREET,NY
 STERLING PLACE,NY
 STUYVESANT AVENUE,NY
 SULLIVAN STREET,NY
 SUMMIT AVENUE,NY
 SUTTON PLACE SOUTH,NY
 SUTTON PLACE,NY
 THIRD AVENUE,NY
 TOMPKINS AVENUE,NY
 UNDERHILL AVENUE,NY
 UNION STREET,NY
 UNITED NATIONS PLAZA,NY
 UNIVERSITY AVENUE,NY
 UTICA AVENUE,NY
 UTOPIA PARKWAY,NY
 VAN BRUNT STREET,NY
 VAN BUREN STREET,NY
 VANDERBILT AVENUE,NY
 VIETOR AVENUE,NY
 W 71ST STREET,NY
 WARREN STREET,NY
 WASHINGTON AVE

In [32]:
location_venues

,Street,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"1 AVENUE,NY",40.770453,-73.954066,NR,40.770082,-73.954978,Cocktail Bar
1,"1 AVENUE,NY",40.770453,-73.954066,Eastside Massage Therapy,40.772103,-73.953681,Massage Studio
2,"1 AVENUE,NY",40.770453,-73.954066,Boqueria,40.771510,-73.956073,Spanish Restaurant
3,"1 AVENUE,NY",40.770453,-73.954066,Oslo Coffee Roasters,40.769364,-73.953598,Coffee Shop
4,"1 AVENUE,NY",40.770453,-73.954066,Agata & Valentina UES Market,40.772002,-73.952997,Grocery Store
5,"1 AVENUE,NY",40.770453,-73.954066,Xi'an Famous Foods,40.772233,-73.954447,Noodle House
6,"1 AVENUE,NY",40.770453,-73.954066,Sushi Ishikawa,40.768741,-73.954126,Sushi Restaurant
7,"1 AVENUE,NY",40.770453,-73.954066,2nd Floor Bar & Essen,40.769687,-73.954350,Cocktail Bar
8,"1 AVENUE,NY",40.770453,-73.954066,Sedutto,40.771760,-73.953148,Ice Cream Shop
9,"1 AVENUE,NY",40.770453,-73.954066,Uva,40.772118,-73.955640,Italian Restaurant


In [33]:
location_venues.groupby('Street').count()

,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Street,,,,,,
"1 AVENUE,NY",100,100,100,100,100,100
"102ND STREET,NY",9,9,9,9,9,9
"103RD STREET,NY",28,28,28,28,28,28
"107TH STREET,NY",9,9,9,9,9,9
"108TH STREET,NY",23,23,23,23,23,23
"10TH AVENUE,NY",14,14,14,14,14,14
"10TH STREET,NY",35,35,35,35,35,35
"11TH AVENUE,NY",18,18,18,18,18,18
"11TH STREET,NY",16,16,16,16,16,16


In [34]:
# The List of Unique Categories
print('There are {} uniques categories.'.format(len(location_venues['Venue Category'].unique())))

There are 483 uniques categories.


In [35]:
location_venues.shape

(25422, 7)

In [36]:
venues_onehot = pd.get_dummies(location_venues[['Venue Category']], prefix="", prefix_sep="")

# adding street column back to dataframe
venues_onehot['Street'] = location_venues['Street'] 

# moving street column to the first column
fixed_columns = [venues_onehot.columns[-1]] + list(venues_onehot.columns[:-1])

#fixed_columns
venues_onehot = venues_onehot[fixed_columns]
venues_onehot.head()

,Street,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,...,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yemeni Restaurant,Yoga Studio,Zoo,Zoo Exhibit
0,"1 AVENUE,NY",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"1 AVENUE,NY",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"1 AVENUE,NY",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"1 AVENUE,NY",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"1 AVENUE,NY",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
nyc_grouped = venues_onehot.groupby('Street').mean().reset_index()
nyc_grouped

,Street,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,...,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yemeni Restaurant,Yoga Studio,Zoo,Zoo Exhibit
0,"1 AVENUE,NY",0.0,0.000000,0.0,0.000000,0.0,0.0,0.010000,0.0,0.0,...,0.0,0.0,0.010000,0.000000,0.000000,0.00,0.0,0.020000,0.000000,0.000000
1,"102ND STREET,NY",0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000
2,"103RD STREET,NY",0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.035714,0.000000,0.00,0.0,0.000000,0.000000,0.000000
3,"107TH STREET,NY",0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000
4,"108TH STREET,NY",0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.043478,0.043478
5,"10TH AVENUE,NY",0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000
6,"10TH STREET,NY",0.0,0.000000,0.0,0.000000,0.0,0.0,0.057143,0.0,0.0,...,0.0,0.0,0.028571,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000
7,"11TH AVENUE,NY",0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000
8,"11TH STREET,NY",0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.062500,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000
9,"12TH AVENUE,NY",0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000


In [38]:
nyc_grouped.shape

(580, 484)

In [39]:
#The top 5 venues/facilities nearby profitable real estate coordinates for each street:

num_top_venues = 5

for hood in nyc_grouped['Street']:
    print("----"+hood+"----")
    temp = nyc_grouped[nyc_grouped['Street'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

---- 1 AVENUE,NY----
                venue  freq
0  Italian Restaurant  0.10
1    Sushi Restaurant  0.05
2         Pizza Place  0.04
3         Coffee Shop  0.03
4        Burger Joint  0.03


---- 102ND STREET,NY----
                 venue  freq
0       Baseball Field  0.11
1       Discount Store  0.11
2            Gun Range  0.11
3         Concert Hall  0.11
4  Fried Chicken Joint  0.11


---- 103RD STREET,NY----
                    venue  freq
0          Baseball Field  0.11
1                    Park  0.11
2                  Bridge  0.07
3  Furniture / Home Store  0.04
4               Bike Shop  0.04


---- 107TH STREET,NY----
               venue  freq
0     Baseball Field  0.22
1         Restaurant  0.11
2        Pizza Place  0.11
3         Playground  0.11
4  Food & Drink Shop  0.11


---- 108TH STREET,NY----
                 venue  freq
0                 Café  0.09
1   Mexican Restaurant  0.09
2                 Park  0.09
3          Zoo Exhibit  0.04
4  Empanada Restaurant  0.04



                venue  freq
0          Playground  0.12
1        Intersection  0.06
2  Italian Restaurant  0.06
3        Liquor Store  0.06
4      Ice Cream Shop  0.06


---- 20TH AVENUE,NY----
                venue  freq
0  Chinese Restaurant  0.12
1              Bakery  0.09
2            Pharmacy  0.09
3          Bagel Shop  0.09
4                Bank  0.06


---- 214TH STREET,NY----
                venue  freq
0         Pizza Place  0.14
1          Food Truck  0.14
2  Falafel Restaurant  0.07
3       Grocery Store  0.07
4  Chinese Restaurant  0.07


---- 215TH STREET,NY----
                venue  freq
0                Café  0.12
1                Park  0.12
2  Mexican Restaurant  0.08
3         Pizza Place  0.04
4  Athletics & Sports  0.04


---- 21ST AVENUE,NY----
               venue  freq
0     Discount Store  0.25
1              Beach  0.25
2        Golf Course  0.25
3  Convenience Store  0.25
4                ATM  0.00


---- 21ST STREET,NY----
                 venue  freq
0    

                  venue  freq
0               Brewery  0.06
1                  Café  0.06
2                 Hotel  0.04
3     Indian Restaurant  0.04
4  Gym / Fitness Center  0.04


---- 44TH STREET,NY----
                  venue  freq
0   Fried Chicken Joint  0.15
1           Video Store  0.15
2  Fast Food Restaurant  0.08
3     Convenience Store  0.08
4        Sandwich Place  0.08


---- 45TH AVENUE,NY----
               venue  freq
0  Korean Restaurant  0.12
1             Lounge  0.06
2        Flower Shop  0.06
3       Noodle House  0.03
4        Pizza Place  0.03


---- 45TH ROAD,NY----
               venue  freq
0  Korean Restaurant  0.16
1   Greek Restaurant  0.06
2               Café  0.04
3        Bus Station  0.04
4             Lounge  0.04


---- 45TH STREET,NY----
                       venue  freq
0                     Bakery  0.11
1  Latin American Restaurant  0.09
2                Pizza Place  0.07
3         Chinese Restaurant  0.07
4                       Bank  0.04


--

---- 69TH STREET,NY----
                 venue  freq
0    Indian Restaurant  0.17
1        Grocery Store  0.11
2           Food Truck  0.11
3  Filipino Restaurant  0.09
4      Thai Restaurant  0.09


---- 6TH AVENUE,NY----
                 venue  freq
0    Korean Restaurant  0.18
1                Hotel  0.07
2  Japanese Restaurant  0.04
3         Burger Joint  0.04
4       Cosmetics Shop  0.04


---- 7 AVENUE,NY----
                venue  freq
0             Theater  0.11
1               Hotel  0.07
2         Coffee Shop  0.04
3  Italian Restaurant  0.04
4        Concert Hall  0.04


---- 70TH STREET,NY----
                venue  freq
0               Track   0.2
1  Italian Restaurant   0.2
2        Skating Rink   0.2
3          Playground   0.2
4                Park   0.2


---- 71ST STREET,NY----
                 venue  freq
0        Deli / Bodega  0.12
1     Department Store  0.08
2               Bakery  0.08
3     Storage Facility  0.04
4  Fried Chicken Joint  0.04


---- 72ND STREET

                venue  freq
0                Café  0.04
1  Mexican Restaurant  0.04
2                 Bar  0.04
3              Bakery  0.04
4         Pizza Place  0.03


---- ATTORNEY STREET,NY----
              venue  freq
0        Hobby Shop  0.17
1            Bakery  0.17
2  Business Service  0.17
3       Golf Course  0.17
4        Food Court  0.17


---- AVENUE D,NY----
            venue  freq
0   Metro Station  0.10
1     Art Gallery  0.05
2  Farmers Market  0.05
3  Baseball Field  0.05
4             Bar  0.05


---- AVENUE I,NY----
              venue  freq
0              Food   0.5
1  Asian Restaurant   0.5
2               ATM   0.0
3          Pharmacy   0.0
4         Pet Store   0.0


---- AVENUE L,NY----
                 venue  freq
0          Pizza Place  0.12
1       Ice Cream Shop  0.12
2  Japanese Restaurant  0.08
3        Grocery Store  0.08
4   Chinese Restaurant  0.08


---- AVENUE M,NY----
              venue  freq
0    Discount Store  0.18
1       Pizza Place  0.18
2 

...

                 venue  freq
0  American Restaurant  0.08
1     Greek Restaurant  0.05
2                 Park  0.05
3             Wine Bar  0.05
4           Skate Park  0.03


---- KOSCIUSZKO STREET,NY----
                  venue  freq
0  Gym / Fitness Center  0.05
1                  Café  0.05
2    Chinese Restaurant  0.05
3      Asian Restaurant  0.05
4     Korean Restaurant  0.05


---- LAFAYETTE AVENUE,NY----
                  venue  freq
0  Caribbean Restaurant  0.10
1                Bakery  0.06
2         Deli / Bodega  0.06
3          Liquor Store  0.06
4                  Café  0.06


---- LAFAYETTE STREET,NY----
                        venue  freq
0                    Pharmacy  0.12
1                        Park  0.06
2                  Food Truck  0.06
3  Tourist Information Center  0.06
4                       Motel  0.06


---- LAIGHT STREET,NY----
                venue  freq
0  Chinese Restaurant  0.15
1       Deli / Bodega  0.08
2                 Pub  0.08
3         Music 

                      venue  freq
0             Deli / Bodega  0.17
1  Mediterranean Restaurant  0.17
2                 Gastropub  0.17
3                      Bank  0.17
4                    Bakery  0.17


---- OCEAN PARKWAY,NY----
               venue  freq
0       Home Service  0.18
1  German Restaurant  0.09
2  Indian Restaurant  0.09
3        Pizza Place  0.09
4               Food  0.09


---- OLD FULTON STREET,NY----
                       venue  freq
0              Deli / Bodega  0.10
1                Coffee Shop  0.07
2  Latin American Restaurant  0.06
3         Chinese Restaurant  0.04
4         Mexican Restaurant  0.04


---- OLINVILLE AVENUE,NY----
                  venue  freq
0    Italian Restaurant  0.04
1  Gym / Fitness Center  0.04
2   Japanese Restaurant  0.03
3             Wine Shop  0.03
4        Cosmetics Shop  0.03


---- ORCHARD STREET,NY----
                 venue  freq
0  Japanese Restaurant  0.05
1          Coffee Shop  0.05
2               Bakery  0.04
3       

          venue  freq
0  Dessert Shop  0.33
1           Spa  0.33
2           Bar  0.33
3           ATM  0.00
4          Park  0.00


---- ST MARKS PLACE,NY----
                           venue  freq
0                     Restaurant  0.17
1               Spiritual Center  0.17
2                    Pizza Place  0.17
3  Vegetarian / Vegan Restaurant  0.17
4            Arts & Crafts Store  0.17


---- ST NICHOLAS AVENUE,NY----
                venue  freq
0                Bank  0.11
1      Clothing Store  0.07
2         Coffee Shop  0.07
3                Café  0.07
4  Italian Restaurant  0.07


---- ST. JOHNS PLACE,NY----
                  venue  freq
0        History Museum   0.5
1             Surf Spot   0.5
2            Non-Profit   0.0
3  Pakistani Restaurant   0.0
4              Pharmacy   0.0


---- STATE STREET,NY----
        venue  freq
0   Speakeasy   1.0
1         ATM   0.0
2  Non-Profit   0.0
3    Pharmacy   0.0
4   Pet Store   0.0


---- STERLING PLACE,NY----
                 v

                          venue  freq
0              Baseball Stadium  0.09
1          Caribbean Restaurant  0.09
2           Monument / Landmark  0.09
3  Theme Park Ride / Attraction  0.09
4                  Skating Rink  0.05


---- WEST 13TH STREET,NY----
                venue  freq
0              Garden  0.25
1        Home Service  0.25
2  Italian Restaurant  0.25
3                Park  0.25
4   Paella Restaurant  0.00


---- WEST 141 STREET,NY----
                           venue  freq
0                         Lounge  0.33
1                     Playground  0.33
2                           Park  0.33
3                            ATM  0.00
4  Paper / Office Supplies Store  0.00


---- WEST 15TH STREET,NY----
       venue  freq
0       Park   1.0
1        ATM   0.0
2  Rest Area   0.0
3   Pharmacy   0.0
4  Pet Store   0.0


---- WEST 16TH STREET,NY----
                     venue  freq
0       Italian Restaurant  0.06
1     Gym / Fitness Center  0.06
2  New American Restaurant  0.05
3

            venue  freq
0         Exhibit  0.15
1            Park  0.08
2  History Museum  0.08
3      Playground  0.04
4     Planetarium  0.03


---- WEST 74TH STREET,NY----
                venue  freq
0          Playground  0.06
1                Park  0.05
2         Coffee Shop  0.05
3  Italian Restaurant  0.05
4            Wine Bar  0.05


---- WEST 75TH STREET,NY----
                           venue  freq
0             Italian Restaurant  0.10
1                    Pizza Place  0.04
2                           Café  0.04
3                            Gym  0.03
4  Vegetarian / Vegan Restaurant  0.03


---- WEST 76TH STREET,NY----
                venue  freq
0  Italian Restaurant  0.08
1       Grocery Store  0.04
2                Park  0.04
3         Coffee Shop  0.03
4            Wine Bar  0.03


---- WEST 77TH STREET,NY----
                venue  freq
0  Italian Restaurant  0.09
1                Café  0.04
2         Coffee Shop  0.04
3         Pizza Place  0.04
4            Pharmacy 

In [40]:
# Defining a function to return the most common venues nearby given coordinates:

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


In [41]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Street']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

In [42]:
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Street'] = nyc_grouped['Street']

for ind in np.arange(nyc_grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(nyc_grouped.iloc[ind, :], num_top_venues)

In [43]:
venues_sorted.head()

,Street,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"1 AVENUE,NY",Italian Restaurant,Sushi Restaurant,Pizza Place,Café,Burger Joint,Cocktail Bar,Coffee Shop,Gym,Spanish Restaurant,Mexican Restaurant
1,"102ND STREET,NY",Skating Rink,Irish Pub,Fried Chicken Joint,Gun Range,Baseball Field,Park,Discount Store,Concert Hall,Bed & Breakfast,Field
2,"103RD STREET,NY",Baseball Field,Park,Bridge,Dog Run,Furniture / Home Store,Bus Line,Supermarket,Café,Bike Shop,Grocery Store
3,"107TH STREET,NY",Baseball Field,Hostel,Skating Rink,Restaurant,Hookah Bar,Pizza Place,Playground,Food & Drink Shop,Farmers Market,Field
4,"108TH STREET,NY",Café,Park,Mexican Restaurant,Zoo Exhibit,Donut Shop,Supermarket,South American Restaurant,Theme Park Ride / Attraction,Gym,Chinese Restaurant


In [44]:
venues_sorted.shape

(580, 11)

In [45]:
nyc_grouped.shape

(580, 484)

In [46]:
nyc_grouped=df

In [47]:

# setting number of clusters
kclusters = 5

nyc_grouped_clustering = nyc_grouped.drop('Street', 1)

# running k-means algorithm clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(nyc_grouped_clustering)

# checking cluster labels generated for each row in the dataframe
kmeans.labels_[0:50]

array([3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 0, 0, 3, 3, 3, 3, 3, 3, 0, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 0, 3, 3, 3, 3, 3, 0, 0, 3, 0,
       3, 3, 0, 3, 3, 3], dtype=int32)

In [48]:
#Dataframe to include Clusters
nyc_grouped_clustering=df
nyc_grouped_clustering.head()


,Street,price,Latitude,Longitude
0,"1 AVENUE,NY",2.201636e+06,40.770453,-73.954066
1,"102ND STREET,NY",6.060346e+05,42.756253,-73.679619
2,"103RD STREET,NY",1.394267e+06,40.796130,-73.961455
3,"107TH STREET,NY",7.456809e+05,42.763483,-73.675924
4,"108TH STREET,NY",7.949143e+05,40.742773,-73.854654


In [49]:

nyc_grouped_clustering.shape

(591, 4)

In [50]:
df.shape

(591, 4)

In [51]:
nyc_grouped_clustering.dtypes

Street        object
price        float64
Latitude     float64
Longitude    float64
dtype: object

In [52]:

df.dtypes

Street        object
price        float64
Latitude     float64
Longitude    float64
dtype: object

In [53]:
# add clustering labels
nyc_grouped_clustering['Cluster Labels'] = kmeans.labels_

# merge nyc_grouped with nyc_data to add latitude/longitude for each neighborhood
nyc_grouped_clustering = nyc_grouped_clustering.join(venues_sorted.set_index('Street'), on='Street')

nyc_grouped_clustering.head(30) # check the last columns!


,Street,price,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"1 AVENUE,NY",2.201636e+06,40.770453,-73.954066,3,Italian Restaurant,Sushi Restaurant,Pizza Place,Café,Burger Joint,Cocktail Bar,Coffee Shop,Gym,Spanish Restaurant,Mexican Restaurant
1,"102ND STREET,NY",6.060346e+05,42.756253,-73.679619,3,Skating Rink,Irish Pub,Fried Chicken Joint,Gun Range,Baseball Field,Park,Discount Store,Concert Hall,Bed & Breakfast,Field
2,"103RD STREET,NY",1.394267e+06,40.796130,-73.961455,3,Baseball Field,Park,Bridge,Dog Run,Furniture / Home Store,Bus Line,Supermarket,Café,Bike Shop,Grocery Store
3,"107TH STREET,NY",7.456809e+05,42.763483,-73.675924,3,Baseball Field,Hostel,Skating Rink,Restaurant,Hookah Bar,Pizza Place,Playground,Food & Drink Shop,Farmers Market,Field
4,"108TH STREET,NY",7.949143e+05,40.742773,-73.854654,3,Café,Park,Mexican Restaurant,Zoo Exhibit,Donut Shop,Supermarket,South American Restaurant,Theme Park Ride / Attraction,Gym,Chinese Restaurant
5,"10TH AVENUE,NY",2.050444e+06,40.724670,-73.690318,3,Pharmacy,Sushi Restaurant,Pet Store,Park,Bagel Shop,Pizza Place,Playground,Fish Market,Pool,Asian Restaurant
6,"10TH STREET,NY",1.264963e+06,40.728657,-73.640106,3,Sushi Restaurant,Golf Course,American Restaurant,Spa,Hotel,Gym / Fitness Center,Hotel Pool,Bank,Bakery,Bagel Shop
7,"11TH AVENUE,NY",1.057836e+06,40.723987,-73.690423,3,Ice Cream Shop,Pet Store,Sushi Restaurant,Park,Pharmacy,Bank,Bagel Shop,Pizza Place,Fish Market,Asian Restaurant
8,"11TH STREET,NY",2.893165e+06,40.730392,-73.639864,0,Sushi Restaurant,Bagel Shop,Department Store,Pizza Place,Grocery Store,Spa,Gym,Soup Place,Ice Cream Shop,Tapas Restaurant
9,"12TH AVENUE,NY",9.901825e+05,40.849118,-73.651344,3,Beach,Park,Pub,Waterfront,Scenic Lookout,Bar,Sushi Restaurant,Farm,Farmers Market,Falafel Restaurant


In [54]:
# Create Map

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = ["blue","red","orange","grey","black"]
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(nyc_grouped_clustering['Latitude'], nyc_grouped_clustering['Longitude'], nyc_grouped_clustering['Street'], nyc_grouped_clustering['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [55]:
nyc_grouped_clustering.loc[nyc_grouped_clustering['Cluster Labels'] == 0, nyc_grouped_clustering.columns[[1] + list(range(5, nyc_grouped_clustering.shape[1]))]].head()

,price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,2.893165e+06,Sushi Restaurant,Bagel Shop,Department Store,Pizza Place,Grocery Store,Spa,Gym,Soup Place,Ice Cream Shop,Tapas Restaurant
11,3.663215e+06,Pharmacy,Massage Studio,Bakery,Deli / Bodega,Shipping Store,Construction & Landscaping,Farmers Market,Film Studio,Filipino Restaurant,Field
12,3.733900e+06,Spa,Bakery,Snack Place,Pharmacy,Steakhouse,Department Store,Gym,Fast Food Restaurant,Film Studio,Filipino Restaurant
19,4.867260e+06,Basketball Court,Sporting Goods Shop,Bar,Scenic Lookout,Cantonese Restaurant,Gourmet Shop,Latin American Restaurant,Food Truck,Baseball Field,Noodle House
34,2.565218e+06,Bus Station,Deli / Bodega,Sandwich Place,Discount Store,Fried Chicken Joint,Gym,Caribbean Restaurant,Liquor Store,Southern / Soul Food Restaurant,Donut Shop


In [56]:
nyc_grouped_clustering.loc[nyc_grouped_clustering['Cluster Labels'] == 1, nyc_grouped_clustering.columns[[1] + list(range(5, nyc_grouped_clustering.shape[1]))]].head()

,price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
30,8.743069e+06,Coffee Shop,Sandwich Place,Martial Arts School,Gym,Wine Shop,Bowling Alley,Mexican Restaurant,Zoo Exhibit,Field,Film Studio
67,1.001298e+07,Pizza Place,Bakery,Chinese Restaurant,Grocery Store,Mexican Restaurant,Italian Restaurant,Pub,Coffee Shop,Café,Greek Restaurant
70,9.478628e+06,Korean Restaurant,Hotel,Café,Gym / Fitness Center,Bar,Spa,Italian Restaurant,Coffee Shop,Bakery,Japanese Restaurant
90,7.031806e+06,Fried Chicken Joint,Video Store,Sandwich Place,Convenience Store,Bar,Fast Food Restaurant,Electronics Store,Greek Restaurant,Donut Shop,Chinese Restaurant
96,8.427354e+06,Italian Restaurant,Café,Japanese Restaurant,Food Truck,Brewery,Bar,Coffee Shop,Deli / Bodega,Dog Run,Park


In [57]:
nyc_grouped_clustering.loc[nyc_grouped_clustering['Cluster Labels'] == 2, nyc_grouped_clustering.columns[[1] + list(range(5, nyc_grouped_clustering.shape[1]))]].head()

,price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
457,38744499.0,Park,Coffee Shop,Hotel,Pizza Place,Gym,Memorial Site,Gourmet Shop,Sandwich Place,Plaza,Food Truck


In [58]:
nyc_grouped_clustering.loc[nyc_grouped_clustering['Cluster Labels'] == 3, nyc_grouped_clustering.columns[[1] + list(range(5, nyc_grouped_clustering.shape[1]))]].head()


,price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,2.201636e+06,Italian Restaurant,Sushi Restaurant,Pizza Place,Café,Burger Joint,Cocktail Bar,Coffee Shop,Gym,Spanish Restaurant,Mexican Restaurant
1,6.060346e+05,Skating Rink,Irish Pub,Fried Chicken Joint,Gun Range,Baseball Field,Park,Discount Store,Concert Hall,Bed & Breakfast,Field
2,1.394267e+06,Baseball Field,Park,Bridge,Dog Run,Furniture / Home Store,Bus Line,Supermarket,Café,Bike Shop,Grocery Store
3,7.456809e+05,Baseball Field,Hostel,Skating Rink,Restaurant,Hookah Bar,Pizza Place,Playground,Food & Drink Shop,Farmers Market,Field
4,7.949143e+05,Café,Park,Mexican Restaurant,Zoo Exhibit,Donut Shop,Supermarket,South American Restaurant,Theme Park Ride / Attraction,Gym,Chinese Restaurant


In [59]:
nyc_grouped_clustering.loc[nyc_grouped_clustering['Cluster Labels'] == 4, nyc_grouped_clustering.columns[[1] + list(range(5, nyc_grouped_clustering.shape[1]))]].head()

,price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
157,2.274096e+07,Deli / Bodega,Pizza Place,Rental Car Location,Music Venue,Playground,Sandwich Place,Shoe Store,Grocery Store,Nightclub,Department Store
352,1.624153e+07,Italian Restaurant,Clothing Store,Mediterranean Restaurant,Shoe Store,Boutique,Gym,Coffee Shop,Bakery,Hotel,Sushi Restaurant
477,1.461789e+07,Sandwich Place,Sushi Restaurant,Italian Restaurant,Steakhouse,Dance Studio,Wings Joint,Grocery Store,Bagel Shop,Bowling Alley,Mexican Restaurant
535,1.960743e+07,Pier,Art Gallery,Boat or Ferry,Dog Run,Spa,Gym,Theater,Wine Shop,Italian Restaurant,Mexican Restaurant
576,1.487835e+07,Diner,Shoe Store,Mobile Phone Shop,Furniture / Home Store,Bank,Laundromat,Mexican Restaurant,BBQ Joint,Coffee Shop,Bagel Shop
